Классификатор C4.5 и три его модификации: с оптимизацией гиперпараметра m, гиперпараметра cf и с одновременной оптимизацией обоих гиперпараметров. Эти четыре классификатора сравнивались на 14 наборах данных. На каждом датасете был посчитан AUC каждого классификатора. Данные записаны в файле:
AUCs.txt

Используя критерий знаковых рангов, проведите попарное сравнение каждого классификатора с каждым. Выберите два классификатора, различие между которыми наиболее статистически значимо. 

In [1]:
import numpy as np
import pandas as pd

data = pd.read_csv("./AUCs.txt", "\t")
data.head()

,Unnamed: 0,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


In [2]:
data.shape

(14, 5)

In [35]:
from itertools import combinations
from scipy import stats

combs = combinations(data.columns[1:], 2)

pairs_p = dict()
for pair in combs:
    pairs_p[pair[0] + ' ' + pair[1]] = stats.wilcoxon(data[pair[0]] - data[pair[1]])
    
p_df = pd.DataFrame(data=pairs_p)
p_df.head()

,C4.5 C4.5+cf,C4.5 C4.5+m,C4.5 C4.5+m+cf,C4.5+cf C4.5+m+cf,C4.5+m C4.5+cf,C4.5+m C4.5+m+cf
0,43.000000,6.500000,11.000000,10.000000,17.000000,22.000000
1,0.861262,0.010757,0.015906,0.022909,0.046333,0.327826


In [40]:
p_df.iloc[1]

(6,)

Сравнивая 4 классификатора между собой, мы проверили 6 гипотез. Давайте сделаем поправку на множественную проверку. Начнём с метода Холма. Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки этим методом?

In [41]:
from statsmodels.sandbox.stats.multicomp import multipletests 

reject, p_corrected, a1, a2 = multipletests(p_df.iloc[1], 
                                            alpha = 0.05, 
                                            method = 'holm') 

In [43]:
p_corrected > 0.05

array([ True,  True,  True,  True,  True,  True])

In [44]:
reject, p_corrected, a1, a2 = multipletests(p_df.iloc[1], 
                                            alpha = 0.05, 
                                            method = 'fdr_bh') 

In [45]:
p_corrected

array([0.86126233, 0.0458182 , 0.0458182 , 0.0458182 , 0.06949909,
       0.39339081])